In [1]:
using Distributed
addprocs(44);

### Fitting a choice model.

First you have to include the necessary package

In [2]:
using pulse_input_DDM, Flatten

┌ Info: Recompiling stale cache file /usr/people/briandd/.julia/compiled/v1.2/pulse_input_DDM/m8WoA.ji for pulse_input_DDM [c2542da2-1f7d-11e9-2506-87691095ebb2]
└ @ Base loading.jl:1240


### Geneerate some data
Blah blah blah

In [3]:
θ_syn = θchoice(θz=θz(σ2_i = 1., B = 13., λ = -0.5, σ2_a = 10., σ2_s = 1.0,
    ϕ = 0.4, τ_ϕ = 0.02), bias=0.1, lapse=0.1);

In [4]:
_, data = synthetic_data(;θ=θ_syn, ntrials=20_000, rng=1, dt=2e-2);

In [5]:
n = 53

53

In [6]:
loglikelihood(θ_syn, data, n)

-7696.220939998316

In [12]:
#x0 = collect(Flatten.flatten(θ_syn));
x0 = vcat([0.1, 15., -0.1, 20., 0.5, 0.2, 0.008], [0.,0.01]) #random initial state

In [24]:
options = choiceoptions(x0=x0)

choiceoptions
  fit: Array{Bool}((9,)) Bool[1, 1, 1, 1, 1, 1, 1, 1, 1]
  lb: Array{Float64}((9,)) [0.0, 8.0, -5.0, 0.0, 0.0, 0.01, 0.005, -30.0, 0.0]
  ub: Array{Float64}((9,)) [2.0, 30.0, 5.0, 100.0, 2.5, 1.2, 1.0, 30.0, 1.0]
  x0: Array{Float64}((9,)) [0.1, 15.0, -0.1, 20.0, 0.5, 0.2, 0.008, 0.0, 0.01]


### Optimize stuff
Blah blah blah

In [25]:
model = optimize(data, options, n)

Fminbox
-------
Initial mu = 0.428371

Fminbox iteration 1
-------------------
Calling inner optimizer with mu = 0.428371

(numbers below include barrier contribution)
Iter     Function value   Gradient norm 
     0     9.212963e+03     1.353167e+05
 * time: 3.1948089599609375e-5
     1     9.112715e+03     2.895310e+04
 * time: 32.75376510620117
     2     8.157816e+03     2.859493e+04
 * time: 45.62749004364014
     3     7.879931e+03     1.280273e+04
 * time: 58.42168998718262
     4     7.873706e+03     1.323454e+04
 * time: 77.75011205673218
     5     7.856478e+03     1.204454e+04
 * time: 90.75878190994263
     6     7.856133e+03     1.191773e+04
 * time: 110.43634104728699
     7     7.841525e+03     9.041094e+03
 * time: 123.47024893760681
     8     7.832685e+03     8.277498e+03
 * time: 136.6883089542389
     9     7.800460e+03     4.103174e+03
 * time: 149.7026629447937
    10     7.763240e+03     6.055365e+03
 * time: 162.56869101524353
    11     7.761642e+03     6.235206

choiceDDM{θchoice{θz{Float64},Float64},Array{choicedata{pulse_input_DDM.choiceinputs{pulse_input_DDM.clicks,pulse_input_DDM.binned_clicks}},1}}
  θ: θchoice{θz{Float64},Float64}
  data: Array{choicedata{pulse_input_DDM.choiceinputs{pulse_input_DDM.clicks,pulse_input_DDM.binned_clicks}}}((20000,))


### Compute Hessian
Blah blah blah

In [ ]:
H = Hessian(model, n)
CI, HPSD = CIs(H);

In [27]:
CI

9-element Array{Float64,1}:
  7.02779325556363    
 27.313530435573018   
  0.32910155780013295 
 22.8607815117612     
  0.7214667685824548  
  0.13990921002103934 
  0.00932707515443111 
  0.1503506830437755  
  0.011360633182694002

In [28]:
hcat(collect(Flatten.flatten(model.θ)) - CI, collect(Flatten.flatten(θ_syn)), collect(Flatten.flatten(model.θ)) + CI)

9×3 Array{Float64,2}:
  -5.92581     1.0    8.12978  
 -13.9408     13.0   40.6863   
  -0.980165   -0.5   -0.321962 
 -14.531      10.0   31.1906   
   0.111395    1.0    1.55433  
   0.299019    0.4    0.578837 
   0.0152466   0.02   0.0339008
  -0.0814441   0.1    0.219257 
   0.0933032   0.1    0.116024 

In [30]:
hcat(collect(Flatten.flatten(model.θ)), collect(Flatten.flatten(θ_syn)))

9×2 Array{Float64,2}:
  1.10198     1.0 
 13.3728     13.0 
 -0.651063   -0.5 
  8.32977    10.0 
  0.832862    1.0 
  0.438928    0.4 
  0.0245737   0.02
  0.0689065   0.1 
  0.104664    0.1 